In [79]:
import numpy as np
import random
from collections import defaultdict

grid_size = 4
actions = ["Up", "Down", "Left", "Right"]

def step(state, action):
    i, j = state
    if action == "Up" and i > 0: i -= 1
    elif action == "Down" and i < grid_size - 1: i += 1
    elif action == "Left" and j > 0: j -= 1
    elif action == "Right" and j < grid_size - 1: j += 1
    
    reward = -1
    done = (i, j) == (grid_size - 1, grid_size - 1)
    death = (i, j) == (1, 1)
    if done:
        reward = 10
    if death:
        reward = -10

    return (i, j), reward, done

def random_policy(state):
    return random.choice(actions)

def random_state():
    i = np.random.randint(0, grid_size)
    j = np.random.randint(0, grid_size)
    state = (i, j)

    while state in [(grid_size-1, grid_size-1), (1, 1)]:
        i = np.random.randint(0, grid_size)
        j = np.random.randint(0, grid_size)
        state = (i, j)
    
    return state

In [80]:
def mc_prediction(policy, num_episodes, gamma=0.9):
    V = defaultdict(float)
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)

    for _ in range(num_episodes):
        episode = []
        state = random_state()
        done = False
        
        while not done:
            action = policy(state)
            next_state, reward, done = step(state, action)
            episode.append((state, action, reward))
            state = next_state

        G = 0
        visited = set()
        for idx, (state, action, reward) in enumerate(episode):
            if state not in visited:
                visited.add(state)
                G = 0
                for k in range(idx, len(episode)):
                    _, _, r = episode[k]
                    G += (gamma ** (k - idx)) * r
                returns_sum[state] += G
                returns_count[state] += 1
                V[state] = returns_sum[state] / returns_count[state]
    
    return V

In [81]:
V = mc_prediction(random_policy, num_episodes=10000)

for i in range(grid_size):
    row = []
    for j in range(grid_size):
        row.append(round(V[(i, j)], 2))
    print(row)

print('\nPolicy:')
for i in range(grid_size):
    row = []
    for j in range(grid_size):
        state = (i, j)

        if state == (grid_size - 1, grid_size - 1):  
            row.append("G")
            continue
        elif state == (1, 1):
            row.append("X")
            continue

        best_val = float("-inf")
        best_move = None
        for a in actions:
            next_state, _, _ = step(state, a)
            if next_state == state: 
                continue
            if next_state == (1,1):
                continue
            if next_state in V and V[next_state] > best_val:
                best_val = V[next_state]
                best_move = a


        row.append(best_move if best_move else "X")
    print(row)


[-15.59, -16.78, -13.13, -10.87]
[-16.93, -14.51, -13.38, -9.14]
[-13.34, -13.36, -8.23, -2.85]
[-10.99, -9.16, -2.71, 0.0]

Policy:
['Right', 'Right', 'Right', 'Down']
['Down', 'X', 'Down', 'Down']
['Down', 'Right', 'Down', 'Down']
['Right', 'Right', 'Right', 'G']
